# get_value_from_dataset

In [1]:
import pandas as pd

# https://fbref.com/en/comps/11/3260/stats/2019-2020-Serie-A-Stats#all_stats_standard
# dataframe with all the main players information for season 2019/20
df = pd.read_csv("dataset2.csv", sep=",")

# a datframe with the some previous information plus the player_overall
price = pd.read_csv("dataset3.csv", sep=",")
df

,Pos.,Giocatore,Nazione,Ruolo,Squadra,Età,Nato,PG,Tit,Min,...,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Partite
0,1,Francesco Acerbi\b96b595c,it ITA,Dif,Lazio,31,1988,36,36,3.162,...,2.0,2.0,1.6,36,6,5,10,6,10,Partite
1,2,Bobby Adekanye\3ed84b2c,nl NED,Att,Lazio,20,1999,11,0,188.000,...,1.7,1.7,0.1,18,82,6,88,82,88,Partite
2,3,Claud Adjapong\ce5b170a,it ITA,Dif,Hellas Verona,21,1998,5,2,231.000,...,0.6,0.6,0.0,6,22,0,22,22,22,Partite
3,4,Lucien Agoume\8bc8829a,fr FRA,Cen,Inter,17,2002,3,0,68.000,...,0.0,0.0,0.1,1,0,8,8,0,8,Partite
4,5,Kevin Agudelo\46fa7204,co COL,CenAtt,Fiorentina,20,1998,3,1,64.000,...,0.3,0.3,0.2,5,45,30,75,45,75,Partite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,Piotr Zieliński\a5296e55,pl POL,Cen,Napoli,25,1994,37,32,2.924,...,4.0,4.0,3.9,78,12,12,24,12,24,Partite
596,597,Jaromír Zmrhal\cbec1196,cz CZE,Cen,Brescia,25,1993,21,11,1.125,...,1.0,1.0,0.5,15,8,4,12,8,12,Partite
597,598,Ervin Zukanović\1253a12e,ba BIH,Dif,SPAL,32,1987,3,2,204.000,...,0.0,0.0,0.0,0,1,0,1,1,1,Partite
598,599,Szymon Żurkowski\4e1d5e59,pl POL,Cen,Fiorentina,21,1997,2,0,12.000,...,0.0,0.0,0.0,0,28,0,28,28,28,Partite


### 1. Get the Probabilities for Age

In [2]:
young = 0
mature = 0
old = 0
age = []

# here we follow the age groups defined in the block 3 of the file "football_player_sale"
for i in df["Età"]:
    if i<24:
        young = young + 1
        age.append("young")
    elif i<31:
        mature = mature + 1
        age.append("mature")
    else:
        old = old + 1
        age.append("old")
total = young + mature + old
young = young / total
mature = mature / total
old = old / total
print(young, mature, old)
df["age"] = age

0.355 0.48 0.165


### 2. Get the expectations for each player

In [3]:
under_expectations = 0
above_expectations = 0
minimum_minuts_played_for_90_min = 10

# we evaluate the goals + assists (or clean sheet) statistics for each player

# also we evaluate the minuts (given by total minuts of the season divided by 90);
# we set the threshold to 90 minuts for season
zipped = zip(df["R + A"].astype(float).div(100), df["npxG+xA"].astype(float).div(100), df["90 min"].astype(float))
expectations = []
for i in zipped:
    if i[0] < i[1]:
        under_expectations = under_expectations + 1
        expectations.append("under_expectations")
    else:
        if i[2] > 1: # threshold of 90 minuts
            above_expectations = above_expectations + 1
            expectations.append("above_expectations")
        else:
            under_expectations = under_expectations + 1
            expectations.append("under_expectations")
under_expectations = format(under_expectations / total, ".3f")
above_expectations = format(above_expectations / total, ".3f")
print(above_expectations, under_expectations)
df["expectations"] = expectations

0.257 0.743


### 3. Get the player_overall probabilities

In [4]:
zipped = zip(df["expectations"], df["age"], price["Overall"], df["Giocatore"])
under_expectations_old = [0,0,0] # bad, normal, top player
above_expectations_old = [0,0,0] # bad, normal, top player
under_expectations_mature = [0,0,0] # bad, normal, top player
above_expectations_mature = [0,0,0] # bad, normal, top player
under_expectations_young = [0,0,0] # bad, normal, top player
above_expectations_young = [0,0,0] # bad, normal, top player

for i in zipped:
    # for each combination, we count if the player is bad, normal or top player
    if i[0] == 'under_expectations' and i[1] == 'old':
        if i[2] == "normal":
            under_expectations_old[1] = under_expectations_old[1] + 1
        elif i[2] == "bad":
            under_expectations_old[0] = under_expectations_old[0] + 1
        else:
            under_expectations_old[2] = under_expectations_old[2] + 1
    
    if i[0] == 'above_expectations' and i[1] == 'old':
        if i[2] == "normal":
            above_expectations_old[1] = above_expectations_old[1] + 1
        elif i[2] == "bad":
            above_expectations_old[0] = above_expectations_old[0] + 1
        else:    
            above_expectations_old[2] = above_expectations_old[2] + 1
    
    if i[0] == 'under_expectations' and i[1] == 'mature':
        if i[2] == "normal":
            under_expectations_mature[1] = under_expectations_mature[1] + 1
        elif i[2] == "bad":
            under_expectations_mature[0] = under_expectations_mature[0] + 1
        else:
            under_expectations_mature[2] = under_expectations_mature[2] + 1
    
    if i[0] == 'above_expectations' and i[1] == 'mature':
        if i[2] == "normal":
            above_expectations_mature[1] = above_expectations_mature[1] + 1
        elif i[2] == "bad":
            above_expectations_mature[0] = above_expectations_mature[0] + 1
        else:
            above_expectations_mature[2] = above_expectations_mature[2] + 1
    
    if i[0] == 'under_expectations' and i[1] == 'young':
        if i[2] == "normal":
            under_expectations_young[1] = under_expectations_young[1] + 1
        elif i[2] == "bad":
            under_expectations_young[0] = under_expectations_young[0] + 1
        else:
            under_expectations_young[2] = under_expectations_young[2] + 1
    
    if i[0] == 'above_expectations' and i[1] == 'young':
        if i[2] == "normal":
            above_expectations_young[1] = above_expectations_young[1] + 1
        elif i[2] == "bad":
            above_expectations_young[0] = above_expectations_young[0] + 1
        else:
            above_expectations_young[2] = above_expectations_young[2] + 1

total = sum(under_expectations_old) + sum(above_expectations_old) + sum(under_expectations_mature) + sum(above_expectations_mature) + sum(under_expectations_young) + sum(above_expectations_young)

# bad, normal, top player
print([format(x / sum(under_expectations_old), ".3f") for x in under_expectations_old])
print([format(x / sum(above_expectations_old), ".3f") for x in above_expectations_old])
print([format(x / sum(under_expectations_mature), ".3f") for x in under_expectations_mature])
print([format(x / sum(above_expectations_mature), ".3f") for x in above_expectations_mature])
print([format(x / sum(under_expectations_young), ".3f") for x in under_expectations_young])
print([format(x / sum(above_expectations_young), ".3f") for x in above_expectations_young])

['0.500', '0.230', '0.270']
['0.400', '0.320', '0.280']
['0.590', '0.257', '0.152']
['0.410', '0.372', '0.218']
['0.759', '0.179', '0.062']
['0.549', '0.333', '0.118']
